In [1]:
import test_config as cfg
import numpy as np
import itertools, functools

%matplotlib inline
import matplotlib.pyplot as plt
import os, os.path
import pickle
import io

import json

import skimage, skimage.io, skimage.transform, skimage.filters
import sklearn, sklearn.metrics

import importlib
import time
import sys
sys.path.append('../src/')
import modutils
import word_processing as wp
import tqdm

In [2]:
%%time
fname = 'D:/Data/bujo_sample_v2/dataset.json'
extraction_path = os.path.join(os.path.dirname(fname),
                               os.path.basename(fname).split('.')[0])
with open(fname, 'r', encoding='utf-8') as f:
    src = json.load(f)
    
words = wp.extract_words_from_dataset(src, (1,))

Wall time: 52 ms


In [3]:
fin_words = [w for w in words if ' ' not in w[0]]

In [4]:
transforms_rotation = wp.generate_rotations(-5, 5, 5)
transforms_cutoff = wp.generate_cutoffs(0.5, 0.9, 3)
combinations_all = wp.generate_combinations(transforms_rotation, transforms_cutoff)
fixed_post = [{'type':'trimx'}, {'type':'trimy'}, {'type':'resize', 'y':32}, {'type':'invert'}]
transforms_all = wp.append_transformations(combinations_all, post_transforms=fixed_post)

In [5]:
train_ids = np.random.choice(np.arange(len(fin_words)), size=int(len(fin_words)*0.6), replace=False)
train_charset = functools.reduce(set.union, [set(fin_words[i][0]) for i in train_ids])
valid_ids = [i for i in range(len(fin_words)) if i not in train_ids]
train_set = [fin_words[i] for i in train_ids]
valid_set = [fin_words[i] for i in valid_ids if set.issubset(set(fin_words[i][0]), train_charset)]

In [12]:
def save_set(path, words, transforms, dst_fname):
    dst_path, dst_name_ext = os.path.split(dst_fname)
    if '.' not in dst_name_ext:
        raise Exception('Not specified extension of file!')
    dst_name = '.'.join(dst_name_ext.split('.')[:-1])
    os.makedirs(os.path.join(dst_path, dst_name), exist_ok=True)
    with open(dst_fname, 'w') as f:
        for (i, (word, fname)) in enumerate(tqdm.tqdm(words)):
            src_image = skimage.io.imread(os.path.join(path, fname), as_grey=True)
            for (j, transform) in enumerate(transforms):
                res_image = wp.perform_transform(src_image, transform)
                loc_fname = 'w{:06d}t{:03d}'.format(i, j)
                res_fname = '{}/{}.png'.format(dst_name, loc_fname)
                f.write('{} {}\n'.format(loc_fname, word))
                skimage.io.imsave(os.path.join(dst_path, res_fname), res_image)

In [ ]:
save_set(extraction_path, train_set, transforms_all, 'D:/Data/HTR/train.txt')

  0%|                                                                                          | 0/933 [00:00<?, ?it/s]c:\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
 12%|█████████▌                                                                      | 111/933 [00:14<02:18,  5.96it/s]c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\train/w000111t000.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\train/w000111t003.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\train/w000111t006.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\t

In [11]:
save_set(extraction_path, valid_set, transforms_all, 'D:/Data/HTR/valid.txt')

  0%|                                                                                          | 0/620 [00:00<?, ?it/s]c:\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
  9%|███████                                                                          | 54/620 [00:08<01:19,  7.14it/s]c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\valid/w000054t000.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\valid/w000054t003.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\valid/w000054t006.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\anaconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: D:/Data/HTR\v